In [16]:
import numpy as np
from scipy.linalg import expm

In [5]:
c_p = 4200  # J/(kg K)

In [4]:
UA     = [   10,     5,    10] # W/K
C      = [40000, 40000, 40000] # J/(kg K)
dot_mi = [    0,     0,     0] # kg/s
dot_mo = [    0,     0,     0] # kg/s
T_inf  = 293.15  # k

N = 3 # number of layers

In [7]:
# mass balance
dot_m = np.zeros((N, N))
# dot_m[ein, aus]

In [ ]:
A = np.zeros((N, N))

A[0, 0]     = (-UA[0] - dot_mo[0]*c_p - dot_m[0, 1]*c_p)/C[0]
A[0, 1]     = (dot_m[1, 0]*c_p)/C[0]

for n in range(1, N-1):
    A[n, n-1] = (dot_m[n-1, n]*c_p)/C[n]
    A[n, n]   = (-UA[n] - dot_mo[n]*c_p - dot_m[n, n+1]*c_p - dot_m[n, n-1]*c_p)/C[n]
    A[n, n+1] = (dot_m[n+1, n]*c_p)/C[n]

A[N-1, N-2] = (dot_m[N-2, N-1]*c_p)/C[N-1]
A[N-1, N-1] = (-UA[N-1] - dot_mo[N-1]*c_p - dot_m[N-1, N-2]*c_p)/C[N-1]

In [9]:
A

array([[-0.00025 ,  0.      ,  0.      ],
       [ 0.      , -0.000125,  0.      ],
       [ 0.      ,  0.      , -0.00025 ]])

In [17]:
delta_t=60
V=0.1
d=400
N=10
U=0.766
k_1=8.2
P_el_nom=2000
h_he=300
T0=[40., 40., 40., 40., 40., 40., 40., 40., 40., 40.]
c_p       = 4200 # J/(kg K)
A_l       = (np.pi * (d/1000)**2)/4 # inside area of the tank in m 
h         = (V*10**9)/((np.pi * d**2)/4) # height of the tank in mm
x_l       = h / (N * 1000) # height of each layer in m
m_l       = (V*1000)/N # mass of water in each layer in kg
UA_m      = U * (np.pi * d * h / N)/10**6 # UA value of middle layers in W / K
UA_a      = U * (np.pi * d * h / N + (d**2/4 * np.pi))/10**6 # UA value of outer layers in W / K
UA        = [UA_a]+[UA_m]*(N-2)+[UA_a] # list of UA values for each layer
C         = [m_l*c_p] * N # J / kg
A         = np.zeros((N, N)) # A matrix
B         = np.zeros((N, N+2)) # B matrix
u         = np.zeros(N+2) # u vector
dot_m     = np.zeros((N, N)) # dot_m[ein, aus]
x         = np.array(T0) + 273.15 # vector containing starting temperatures in K
k         = [  0] *N # starting k values (values will be determined in logic) W/(m K)
k_2       = 999999 # W/(m K)
he_layers = int(np.ceil(h_he/(h/N))) # layers of the tank receiving heat from heating element directly
P_el      = [0] * (N-he_layers) + [P_el_nom/he_layers] * he_layers # list of electrical power input of each layer in W

time=0
dot_m_o_DHW=0.1
T_i_DHW=10
T_inf=10+273.15
state=0

dot_m_i = [             0] + [0]*(N-2) + [   dot_m_o_DHW] # kg/s
T_i     = [             0] + [0]*(N-2) + [T_i_DHW+273.15] # K
dot_m_o = [   dot_m_o_DHW] + [0]*(N-2) + [             0] # kg/s

In [22]:
def logic_heat_transfer(dot_m_i, dot_m_o, dot_m, UA, A, B, u, C, N, c_p, x_prev, T_inf, T_i, delta_t, x_l, k, A_l, k_1, k_2, P_el, state):
    # logic with heat transfer between layers

    #################
    # Effective thermal conductivity
    #################
    
    for n in range(1, N):
        if x_prev[n-1] >= x_prev[n]:
            k[n] = k_1
        elif x_prev[n-1] < x_prev[n]:
            k[n] = k_2

    # check mass balance
    assert sum(dot_m_i) - sum(dot_m_o) == 0, 'mass balance not zero'

    #################
    # Mass balance
    #################
    # Zeroth Layer (Top)
    dot_m_net = dot_m_i[0] - dot_m_o[0]
    if dot_m_net > 0:
        dot_m[0, 1] = dot_m_net
        dot_m[1, 0] = 0
    else:
        dot_m[0, 1] = 0
        dot_m[1, 0] = -dot_m_net

    # Mid Layers
    for n in range(1, N-1):
        dot_m_net = dot_m_i[n] - dot_m_o[n] + dot_m[n-1, n] - dot_m[n, n-1]
        if dot_m_net > 0:
            dot_m[n, n+1] = dot_m_net
            dot_m[n+1, n] = 0
        else:
            dot_m[n, n+1] = 0
            dot_m[n+1, n] = -dot_m_net

    #################
    # A Matrix
    #################
    # Zeroth Layer (Top)
    A[0, 0]     = (-UA[0] - dot_m_o[0]*c_p - dot_m[0, 1]*c_p)/C[0] - k[1]*A_l/(x_l*C[0])
    A[0, 1]     = (dot_m[1, 0]*c_p )/C[0]+ k[1]*A_l/(x_l*C[0])

    # N-2  Mid Layers
    for n in range(1, N-1):
        A[n, n-1] = (dot_m[n-1, n]*c_p)/C[n] + k[n]*A_l/(x_l*C[n])
        A[n, n]   = (-UA[n] - dot_m_o[n]*c_p - dot_m[n, n+1]*c_p - dot_m[n, n-1]*c_p)/C[n] - k[n]*A_l/(x_l*C[n]) - k[n+1]*A_l/(x_l*C[n])
        A[n, n+1] = (dot_m[n+1, n]*c_p)/C[n] + k[n+1]*A_l/(x_l*C[n])

    # N-th Layer (Bottom)
    A[N-1, N-2] = (dot_m[N-2, N-1]*c_p)/C[N-1] + k[N-1]*A_l/(x_l*C[N-1])
    A[N-1, N-1] = (-UA[N-1] - dot_m_o[N-1]*c_p - dot_m[N-1, N-2]*c_p)/C[N-1] - k[N-1]*A_l/(x_l*C[N-1])


    #################
    # B-Vector
    #################
    for n in range(N):
        B[n, 0] = UA[n]/C[n]
        B[n, n+1] = c_p/C[n]
        B[n, N+1] = P_el[n]/C[n]
    
    #################
    # u-Vector
    #################
    u[0] = T_inf
    u[N+1] = state
    for n in range(N):
        u[n+1] = dot_m_i[n]*T_i[n]

    # discretize

    # discretize
    exponent = np.vstack((np.hstack((A, B)), np.zeros((B.shape[1], A.shape[1]+B.shape[1]))))*delta_t

    res = expm(exponent)

    Ad = res[:A.shape[0], :A.shape[1]]
    Bd = res[:B.shape[0], A.shape[1]:]
    
    x=Ad@x_prev+Bd@u.T

    return x

In [21]:
logic_heat_transfer(dot_m_i, dot_m_o, dot_m, UA, A, B, u, C, N, c_p, x, T_inf, T_i, delta_t, x_l, k, A_l, k_1, k_2, P_el, state)

array([313.14363635, 313.14669129, 313.14670835, 313.14660213,
       313.14539723, 313.13369596, 313.03895963, 312.42490931,
       309.43562143, 299.70070811])